<a href="https://colab.research.google.com/github/TheClockworkk/ML-Labs-Homeworks/blob/main/lab1_sgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ТЗ к Лабораторной работе №1:**

1. [x] Выбрать тестовые функции оптимизации (Химмельблау и Розенброк)
2. [x] Запрограммировать собственнуб реализацию классического градиентного спуска
3. [x] Запрограммировать пайплайн тестирования алгоритма оптимизации
   - [x] Визуализации функции и точки оптимума
   - [x] Вычисление погрешности найденного решения в сравнение с аналитическим
   - [x] Визуализации точки найденного решения (можно добавить анимацию на плюс балл)
4. [ ] Запрограммировать метод вычисления градиента
   - [ ] Передача функции градиента от пользователя
   - [x] Символьное вычисление градиента (например с помощью sympy) (на доп балл)
   - [x] Численная аппроксимация градиента (на доп балл)
5. [x] Запрограммировать одну моментную модификацию и протестировать ее
6. [x] Запрограммировать одну адаптивную модификацию и протестировать ее
7. [x] Запрограммировать метод эволюции темпа обучения и/или метод выбора начального приближения и протестировать их


In [37]:
from matplotlib import pyplot as plt
import numpy as np
from numpy import linalg as la
from datetime import datetime
import plotly.graph_objects as go
from sympy import symbols, diff, Matrix

class Himmelblau():
  @staticmethod
  def fuction(x):
        return (x[0] * x[0] + x[1] - 11) ** 2 + (x[0] + x[1] * x[1] - 7) ** 2

  @staticmethod
  def get_bounds():
    return np.array([[-6, -6],
                     [6, 6]])

  @staticmethod
  def get_min():
        return np.array([3., 2., 0.])

  # def gradient(x):
  #       return np.array([4 * x[0] * (x[0] * x[0] + x[1] - 11) + 2 * (x[0] + x[1] * x[1] - 7), 4 * x[1] * (x[1] * x[1] + x[0] - 7) + 2 * (x[1] + x[0] * x[0] - 11)])

  # def numerical_gradient(x, func, h=1e-5):
  #         gradient = np.zeros_like(x, dtype=float)
  #         for i in range(len(x)):
  #             x_plus_h = x.copy()
  #             x_plus_h[i] += h
  #             gradient[i] = (func(x_plus_h) - func(x)) / h
  #         return gradient

  @staticmethod
  def gradient(x):
      x1, x2 = symbols('x1 x2')
      f = (x1**2 + x2 - 11)**2 + (x1 + x2**2 - 7)**2
      df_dx1 = diff(f, x1)
      df_dx2 = diff(f, x2)
      return np.array([float(df_dx1.subs({x1: x[0], x2: x[1]})), float(df_dx2.subs({x1: x[0], x2: x[1]}))])


class Rosenbrock:
    @staticmethod
    def function(x):
        return sum(100.0 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)

    @staticmethod
    def get_bounds():
        return np.array([[-3, -3],
                         [3, 10]])

    @staticmethod
    def get_min():
        return np.array([1., 1., 0.])

    @staticmethod
    def gradient(x):
        grad = np.zeros_like(x)
        grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
        grad[1:-1] = 200 * (x[1:-1] - x[:-2]**2) - 400 * x[1:-1] * (x[2:] - x[1:-1]**2) - 2 * (1 - x[1:-1])
        grad[-1] = 200 * (x[-1] - x[-2]**2)
        return grad



def draw_result(func: callable, bounds: np.array, glob_min: np.array, history: np.array, title: str) -> None:
    x = np.linspace(bounds[0, 0], bounds[1, 0], 100)
    y = np.linspace(bounds[0, 1], bounds[1, 1], 100)

    x_grid, y_grid = np.meshgrid(x, y)
    z_grid = func(np.array([x_grid, y_grid]))

    fig = go.Figure()

    # Plot the surface
    fig.add_trace(go.Surface(x=x, y=y, z=z_grid, colorscale='Viridis', opacity=0.8))

    # Plot the optimization path
    fig.add_trace(go.Scatter3d(x=history[:, 0], y=history[:, 1], z=history[:, 2],
                               mode='lines+markers', marker=dict(size=4, color='black'),
                               line=dict(color='black', width=2), name='Optimization Path'))

    # Plot the start and end points
    fig.add_trace(go.Scatter3d(x=[history[0, 0]], y=[history[0, 1]], z=[history[0, 2]],
                               mode='markers', marker=dict(size=8, color='white', line=dict(color='black', width=2)),
                               name='Start Point'))

    fig.add_trace(go.Scatter3d(x=[history[-1, 0]], y=[history[-1, 1]], z=[history[-1, 2]],
                               mode='markers', marker=dict(size=8, color='white', line=dict(color='black', width=2)),
                               name='End Point'))

    # Set layout and show the figure
    fig.update_layout(scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
                      title=title,
                      showlegend=True,
                      margin=dict(l=0, r=0, b=0, t=40),
                      scene_camera=dict(eye=dict(x=-1.5, y=-1.5, z=1)))

    fig.show()



def my_GD(
    func: callable, grad: callable, start_params: np.ndarray, glob_min: np.ndarray,
    max_iter: int = 1000, lr: float = 0.1, delta: float = 0.001
    ) -> np.array:

    """Классический градиентный спуск

    Args:
        func (callable): функция искуственного ландшафта
        start_params (np.ndarray): стартовый набор параметров
        glob_min (np.ndarray): глобальный минимум рассматриваемой функции
        max_iter (int, optional): ограничение по кол-ву итераций. Defaults to 100.
        lr (float, optional): шаг обучения. Defaults to 0.1.
        delta (float, optional): радиус сходимости. Defaults to 0.001.

    Returns:
        np.array: история градиентного спуска
    """

    # начальный набор параметров
    params = start_params.copy()
    history = [np.array([params[0], params[1], func(params)])]

    step = 0
    while (step < max_iter and la.norm(history[-1] - glob_min) > delta):



        # новое значение параметров
        params = params - lr * grad(params)

        # сохраняем результат
        history.append(np.array([params[0], params[1], func(params)]))
        step += 1

    return np.array(history)





def momentum_GD(
    func: callable, grad: callable, start_params: np.ndarray, glob_min: np.ndarray,
    max_iter: int = 1000, lr: float = 0.1, delta: float = 0.001, beta: float = 0.5
    ) -> np.array:

    """Градиентный спуск с моментной модификацией

    Args:
        func (callable): функция искуственного ландшафта
        start_params (np.ndarray): начальный набор параметров
        glob_min (np.ndarray): глобальный минимум рассматриваемой функции
        max_iter (int, optional): ограничение по кол-ву итераций. Defaults to 1000.
        lr (float, optional): шаг обучения. Defaults to 0.1.
        delta (float, optional): радиус сходимости. Defaults to 0.001.
        beta (float, optional): Коэффициент энертности. Defaults to 0.5.

    Returns:
        np.array: история градиентного спуска
    """

    # начальный набор параметров
    params_prev = start_params.copy()
    params = start_params.copy()
    history = [np.array([params[0], params[1], func(params)])]

    step = 0
    while (step < max_iter and la.norm(history[-1] - glob_min) > delta):

        # новое значение параметров
        params_new = params - lr * grad(params) + beta * (params - params_prev)
        params_prev = params
        params = params_new

        # сохраняем результат
        history.append(np.array([params[0], params[1], func(params)]))
        step += 1

    return np.array(history)





def Adam_GD(
    func: callable, grad: callable, start_params: np.ndarray, glob_min: np.ndarray, max_iter: int = 1000,
    lr: float = 0.1, b1: float = 0.6, b2: float = 0.999, e: float = 10e-8, delta: float = 0.001
    ) -> np.array:

    """Адаптивный градиентный спуск: Adam

    Args:
        func (callable): функция искуственного ландшавта
        start_params (np.ndarray): стартовый набор параметров
        glob_min (np.ndarray): глобальный минимум рассматриваемой функции
        max_iter (int, optional): Ограничение по кол-ву итераций. Defaults to 1000.
        lr (float, optional): шаг обучения. Defaults to 0.1.
        b1 (float, optional): параметр beta1. Defaults to 0.6.
        b2 (float, optional): параметр beta2. Defaults to 0.999.
        e (float, optional): "бесконечно" малое число. Defaults to 10e-8.
        delta (float, optional): радиус сходимости. Defaults to 0.001.

    Returns:
        np.array: история градиентного спуска
    """

    # начальный набор параметров
    params = start_params.copy()
    history = [np.array([params[0], params[1], func(params)])]

    # инициализация моментов
    m = np.array([0, 0])
    v = np.array([0, 0])

    step = 0
    while (step < max_iter and la.norm(history[-1] - glob_min) > delta):

        # считаем моменты
        m = b1 * m + (1 - b1) * grad(params)
        v = b2 * v + (1 - b2) * grad(params) ** 2

        # новое значение параметров
        params = params - lr * m / (np.sqrt(v) + e)

        # сохраняем результат
        history.append(np.array([params[0], params[1], func(params)]))
        step += 1

    return np.array(history)

def my_evolutionary_GD(
    func: callable, grad: callable, start_params: np.ndarray, glob_min: np.ndarray,
    max_iter: int = 1000, lr: float = 0.1, delta: float = 0.001
    ) -> np.array:

    """Градиентный спуск с эволюцией темпа обучения и выбором начального приближения.

    Args:
        func (callable): функция искуственного ландшафта
        grad (callable): функция градиента
        start_params (np.ndarray): начальный набор параметров
        glob_min (np.ndarray): глобальный минимум рассматриваемой функции
        max_iter (int, optional): ограничение по кол-ву итераций. Defaults to 1000.
        lr (float, optional): шаг обучения. Defaults to 0.1.
        delta (float, optional): радиус сходимости. Defaults to 0.001.

    Returns:
        np.array: история градиентного спуска
    """

    # начальный набор параметров
    params = start_params.copy()
    history = [np.array([params[0], params[1], func(params)])]

    step = 0
    while (step < max_iter and np.linalg.norm(history[-1] - glob_min) > delta):

        # новое значение параметров
        params = params - lr * grad(params)
        lr = lr / 2 # обновление темпа обучения

        # сохраняем результат
        history.append(np.array([params[0], params[1], func(params)]))
        step += 1

    return np.array(history)

GD

In [38]:
# Для Химмельблау
start_time = datetime.now()
history = my_GD(Himmelblau.fuction, Himmelblau.gradient, np.array([5,5]), Himmelblau.get_min(), max_iter = 1000, lr = 0.006)
print("Himmelblau default sgd\n", datetime.now() - start_time)
np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
print(f"Starting point:\t\t{history[0]}")
np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
print(f"The minimum found:\t{history[-1]}")
print(f"Global minimum:\t\t{Himmelblau.get_min()}")
print(f"Error:\t\t\t{abs(Himmelblau.get_min()[2] - history[-1])[2]}")
print(f"Number of iterations:\t{len(history)}")
draw_result(Himmelblau.fuction, Himmelblau.get_bounds(), Himmelblau.get_min(), history, "Default GD")

Himmelblau default sgd
 0:00:00.576313
Starting point:		[5.00 5.00 890.00]
The minimum found:	[3.00e+00 2.00e+00 1.25e-05]
Global minimum:		[3.00e+00 2.00e+00 0.00e+00]
Error:			1.2515563024080618e-05
Number of iterations:	35


In [39]:
# Для Розенброка
start_time = datetime.now()
history = my_GD(Rosenbrock.function, Rosenbrock.gradient, np.array([-2.8, -2.1]), Rosenbrock.get_min(), max_iter = 1000, lr = 0.0001)
print("Rosenbrock default sgd\n", datetime.now() - start_time)
np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
print(f"Starting point:\t\t{history[0]}")
np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
print(f"The minimum found:\t{history[-1]}")
print(f"Global minimum:\t\t{Rosenbrock.get_min()}")
print(f"Error:\t\t\t{abs(Rosenbrock.get_min()[2] - history[-1])[2]}")
print(f"Number of iterations:\t{len(history)}")
draw_result(Rosenbrock.function, Rosenbrock.get_bounds(), Rosenbrock.get_min(), history, "Default GD")

Rosenbrock default sgd
 0:00:00.106627
Starting point:		[-2.80 -2.10 9894.80]
The minimum found:	[1.36e-01 1.65e-02 7.46e-01]
Global minimum:		[1.00e+00 1.00e+00 0.00e+00]
Error:			0.746378576932792
Number of iterations:	1001


Моментная модификация GD

In [40]:
# Для Химмельблау
start_time = datetime.now()
history = momentum_GD(Himmelblau.fuction, Himmelblau.gradient, np.array([5,5]), Himmelblau.get_min(), max_iter = 1000, lr = 0.004)
print("Himmelblau inertial sgd\n", datetime.now() - start_time)
np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
print(f"Starting point:\t\t{history[0]}")
np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
print(f"The minimum found:\t{history[-1]}")
print(f"Global minimum:\t\t{Himmelblau.get_min()}")
print(f"Error:\t\t\t{abs(Himmelblau.get_min()[2] - history[-1])[2]}")
print(f"Number of iterations:\t{len(history)}")
draw_result(Himmelblau.fuction, Himmelblau.get_bounds(), Himmelblau.get_min(), history, "Inertial GD")

Himmelblau inertial sgd
 0:00:00.247467
Starting point:		[5.00 5.00 890.00]
The minimum found:	[3.00e+00 2.00e+00 4.13e-06]
Global minimum:		[3.00e+00 2.00e+00 0.00e+00]
Error:			4.13330142026512e-06
Number of iterations:	29


In [41]:
# Для Розенброка
start_time = datetime.now()
history = momentum_GD(Rosenbrock.function, Rosenbrock.gradient, np.array([-2.8, -2.1]), Rosenbrock.get_min(), max_iter = 1000, lr = 0.0001)
print("Rosenbrock inertial sgd\n", datetime.now() - start_time)
np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
print(f"Starting point:\t\t{history[0]}")
np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
print(f"The minimum found:\t{history[-1]}")
print(f"Global minimum:\t\t{Rosenbrock.get_min()}")
print(f"Error:\t\t\t{abs(Rosenbrock.get_min()[2] - history[-1])[2]}")
print(f"Number of iterations:\t{len(history)}")
draw_result(Rosenbrock.function, Rosenbrock.get_bounds(), Rosenbrock.get_min(), history, "Inertial GD")

Rosenbrock inertial sgd
 0:00:00.075945
Starting point:		[-2.80 -2.10 9894.80]
The minimum found:	[2.99e-01 8.66e-02 4.92e-01]
Global minimum:		[1.00e+00 1.00e+00 0.00e+00]
Error:			0.49175650116693587
Number of iterations:	1001


Adam GD

In [42]:
# Для Химмельблау
start_time = datetime.now()
history = Adam_GD(Himmelblau.fuction, Himmelblau.gradient, np.array([5,5]), Himmelblau.get_min(), max_iter = 1000, lr = 0.1)
print("Himmelblau Adam sgd\n", datetime.now() - start_time)
np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
print(f"Starting point:\t\t{history[0]}")
np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
print(f"The minimum found:\t{history[-1]}")
print(f"Global minimum:\t\t{Himmelblau.get_min()}")
print(f"Error:\t\t\t{abs(Himmelblau.get_min()[2] - history[-1])[2]}")
print(f"Number of iterations:\t{len(history)}")
draw_result(Himmelblau.fuction, Himmelblau.get_bounds(), Himmelblau.get_min(), history, "Adam")

Himmelblau Adam sgd
 0:00:00.277595
Starting point:		[5.00 5.00 890.00]
The minimum found:	[3.00e+00 2.00e+00 2.80e-05]
Global minimum:		[3.00e+00 2.00e+00 0.00e+00]
Error:			2.7973426823945723e-05
Number of iterations:	32


In [43]:
# Для Розенброка
start_time = datetime.now()
history = Adam_GD(Rosenbrock.function, Rosenbrock.gradient, np.array([-2.8, -2.1]), Rosenbrock.get_min(), max_iter = 1000, lr = 0.09)
print("Rosenbrock Adam sgd\n", datetime.now() - start_time)
np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
print(f"Starting point:\t\t{history[0]}")
np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
print(f"The minimum found:\t{history[-1]}")
print(f"Global minimum:\t\t{Rosenbrock.get_min()}")
print(f"Error:\t\t\t{abs(Rosenbrock.get_min()[2] - history[-1])[2]}")
print(f"Number of iterations:\t{len(history)}")
draw_result(Rosenbrock.function, Rosenbrock.get_bounds(), Rosenbrock.get_min(), history, "Adam")

Rosenbrock Adam sgd
 0:00:00.150203
Starting point:		[-2.80 -2.10 9894.80]
The minimum found:	[8.57e-01 7.34e-01 2.05e-02]
Global minimum:		[1.00e+00 1.00e+00 0.00e+00]
Error:			0.02054018989222027
Number of iterations:	1001


Метод эволюции темпа обучения

In [44]:
# Для Химмельблау
start_time = datetime.now()
history = my_evolutionary_GD(Himmelblau.fuction, Himmelblau.gradient, np.array([5,5]), Himmelblau.get_min(), max_iter = 1000, lr = 0.006)
print("Himmelblau Evolution sgd\n", datetime.now() - start_time)
np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
print(f"Starting point:\t\t{history[0]}")
np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
print(f"The minimum found:\t{history[-1]}")
print(f"Global minimum:\t\t{Himmelblau.get_min()}")
print(f"Error:\t\t\t{abs(Himmelblau.get_min()[2] - history[-1])[2]}")
print(f"Number of iterations:\t{len(history)}")
draw_result(Himmelblau.fuction, Himmelblau.get_bounds(), Himmelblau.get_min(), history, "Evolution")

Himmelblau Evolution sgd
 0:00:01.648480
Starting point:		[5.00 5.00 890.00]
The minimum found:	[2.61e+00 2.06e+00 4.53e+00]
Global minimum:		[3.00e+00 2.00e+00 0.00e+00]
Error:			4.528573300081343
Number of iterations:	1001


In [45]:
# Для Розенброка
start_time = datetime.now()
history = my_evolutionary_GD(Rosenbrock.function, Rosenbrock.gradient, np.array([-2.8, -2.1]), Rosenbrock.get_min(), max_iter = 1000, lr = 0.0001)
print("Rosenbrock default sgd\n", datetime.now() - start_time)
np.set_printoptions(formatter={'float_kind':"{:.2f}".format})
print(f"Starting point:\t\t{history[0]}")
np.set_printoptions(formatter={'float_kind':"{:.2e}".format})
print(f"The minimum found:\t{history[-1]}")
print(f"Global minimum:\t\t{Rosenbrock.get_min()}")
print(f"Error:\t\t\t{abs(Rosenbrock.get_min()[2] - history[-1])[2]}")
print(f"Number of iterations:\t{len(history)}")
draw_result(Rosenbrock.function, Rosenbrock.get_bounds(), Rosenbrock.get_min(), history, "Evolution")

Rosenbrock default sgd
 0:00:00.057586
Starting point:		[-2.80 -2.10 9894.80]
The minimum found:	[-1.41e+00 -1.81e+00 1.44e+03]
Global minimum:		[1.00e+00 1.00e+00 0.00e+00]
Error:			1440.812908639748
Number of iterations:	1001
